In [1]:
import pandas as pd
import numpy as np

import good_tree

# all imports needed for this sheet

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


%matplotlib inline
import seaborn as sns
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_excel('DAM_Oct10_2020.xlsx')

In [3]:
df = df.drop(columns=['3.1 FirstName', '3.1 LastName', '3.2 SocSecNo', 
                      '3.3 Birthdate', 'V5 Prior Address'])

In [4]:
df.head()

,2.1 Organization Name,2.2 Project Name,2.4 ProjectType,2.5 Utilization Tracking Method (Invalid),2.6 Federal Grant Programs,Enrollment Created By,5.8 Personal ID,5.9 Household ID,3.15 Relationship to HoH,3.16 Client Location,...,4.2.13k Retirement (Social Security),4.2.14l Pension from a Former Job,4.2.15m Child Support,4.2.16n Alimony,4.2.17o Other Income,Chronic Homeless Status_vHMISDatAssessment,Chronic Homeless Status_EvaluatevHMIS&HMISDA,Email,HomePhone,WorkPhone
0,Family Promise of Spokane,FPS--ES--Bridges Family Shelter,Emergency shelter,Entry/Exit (e/e),NaN,Lynnetta Lewis,56550,92775,Son,NaN,...,0,0,0.0,0,0,0,0,NaN,NaN,NaN
1,Family Promise of Spokane,FPS--ES--Bridges Family Shelter,Emergency shelter,Entry/Exit (e/e),NaN,Lynnetta Lewis,36138,92775,Self,Spokane City & County CoC,...,0,0,0.0,0,0,0,0,NaN,509-847-3635,NaN
2,Family Promise of Spokane,FPS--ES--Bridges Family Shelter,Emergency shelter,Entry/Exit (e/e),NaN,Lynnetta Lewis,36139,92775,Daughter,NaN,...,0,0,0.0,0,0,0,0,NaN,NaN,NaN
3,Family Promise of Spokane,FPS--ES--Bridges Family Shelter,Emergency shelter,Entry/Exit (e/e),NaN,Lynnetta Lewis,36140,92775,Son,NaN,...,0,0,0.0,0,0,0,0,NaN,NaN,NaN
4,Family Promise of Spokane,FPS--ES--Bridges Family Shelter,Emergency shelter,Entry/Exit (e/e),NaN,Lynnetta Lewis,36141,92775,Significant Other (Non-Married),NaN,...,0,0,0.0,0,0,1,1,NaN,964-8982,NaN


In [5]:
exit_reasons = ['Rental by client with RRH or equivalent subsidy', 
                'Rental by client, no ongoing housing subsidy', 
                'Staying or living with family, permanent tenure', 
                'Rental by client, other ongoing housing subsidy',
                'Permanent housing (other than RRH) for formerly homeless persons', 
                'Staying or living with friends, permanent tenure', 
                'Owned by client, with ongoing housing subsidy', 
                'Rental by client, VASH housing Subsidy'
               ]

 # create target column (multiple types of exits to perm)
df['perm_leaver'] = df['3.12 Exit Destination'].isin(exit_reasons)

In [6]:
df = df.rename(columns={'2.1 Organization Name':'org_name',
                        '2.2 Project Name':'project_name',
                        '2.4 ProjectType':'project_type',
                        '2.5 Utilization Tracking Method (Invalid)':'util_track_method',
                        '2.6 Federal Grant Programs':'fed_grant_programs',
                        'Enrollment Created By':'enrollment_created_by',
                        '3.1 FirstName':'first_name','3.1 LastName':'last_name',
                        '5.8 Personal ID':'personal_id','5.9 Household ID':'case_id',
                        '3.15 Relationship to HoH':'relationship_to_HoH',
                        '3.16 Client Location':'client_location','CaseMembers':'case_members',
                        '3.10 Enroll Date':'enroll_date','3.11 Exit Date':'exit_date',
                        '3.12 Exit Destination':'exit_destination','3.2 SocSecNo':'ssn',
                        '3.2 Social Security Quality':'ssn_quality','3.3 Birthdate':'dob',
                        '3.3 Birthdate Quality':'dob_quality','Age at Enrollment':'age_at_enrollment',
                        'Current Age':'current_age','3.4 Race':'race','3.5 Ethnicity':'ethnicity',
                        '3.6 Gender':'gender','3.7 Veteran Status':'vet_status',
                        '3.8 Disabling Condition at Entry':'disabling_cond_at_entry',
                        '3.917 Living Situation':'living_situation',
                        '3.917b Stayed Less Than 7 Nights':'stayed_7_or_less',
                        '3.917 Stayed Less Than 90 Days':'stayed_90_or_less',
                        '3.917b Stayed in Streets, ES or SH Night Before':'stayed_street_ES_or_SH_night_before',
                        '3.917 Length of Stay':'length_of_stay',
                        '3.917 Homeless Start Date':'homeless_start_date',
                        'Length of Time Homeless (3.917 Approximate Start)':'length_of_time_homeless',
                        '3.917 Times Homeless Last 3 Years':'times_homeless_last_3years',
                        '3.917 Total Months Homeless Last 3 Years':'total_months_homeless',
                        'V5 Last Permanent Address':'last_perm_address',
                        'V5 Prior Address':'prior_address','V5 State':'state',
                        'V5 Zip':'zip','Municipality (City or County)':'municipality',
                        'Days Enrolled in Project':'days_enrolled_in_project',
                        'RRH In Permanent Housing':'rrh_in_perm_housing',
                        'RRH Date Of Move-In':'rrh_date_of_move-in',
                        'Days Enrolled Until RRH Date of Move-in':'days_enrolled_until_rrh_movein',
                        '4.1 Housing Status':'housing_status',
                        '4.4 Covered by Health Insurance':'covered_by_health_insurance',
                        '4.11 Domestic Violence':'domestic_violence',
                        '4.11 Domestic Violence - Currently Fleeing DV?':'currently_fleeing',
                        '4.11 Domestic Violence - When it Occurred':'when_dv_occured',
                        '4.13 Engagement Date':'engagement_date',
                        'Days Enrolled Until Engagement Date':'days_enrolled_until_engaged',
                        '4.24 Current Status (Retired Data Element)':'current_status',
                        '4.24 In School (Retired Data Element)':'in_school',
                        '4.24 Connected to McKinney Vento Liason (Retired)':'connected_to_MVento',
                        'Household Type':'household_type',
                        'Latitude':'latitude','Longitude':'longitude',
                        'R1 Referral Source':'referal_source',
                        'R2 Date Status Determined':'date_status_determined',
                        'R2 Enroll Status':'enroll_status',
                        'R2 Runaway Youth':'runaway_youth',
                        'R2 Reason Why No Services Funded':'reason_why_no_services_funded',
                        'R3 Sexual Orientation':'sexual_orientation',
                        'R4 Last Grade Completed':'last_grade_completed',
                        'R5 School Status':'school_status',
                        'R6 Employed Status':'employed_status',
                        'R6 Why Not Employed':'reason_not_employed',
                        'R6 Type of Employment':'type_of_employment',
                        'R6 Looking for Work':'looking_for_work',
                        'R7 General Health Status':'general_health_status',
                        'R8 Dental Health Status':'dental_health_status',
                        'R9 Mental Health Status':'mental_health_status',
                        'R10 Pregnancy Status':'pregnancy_status',
                        'R10 Pregnancy Due Date':'pregnancy_due_date',
                        'Client Record Restricted':'client_record_restricted',
                        'InfoReleaseNo':'infoReleaseNo',
                        'Information Release Status':'info_release_status',
                        '4.12 Contact Services':'contact_services',
                        'Date of Last Contact (Beta)':'date_of_last_contact',
                        'Date of First Contact (Beta)':'date_of_first_contact',
                        'Count of Bed Nights (Housing Check-ins)':'housing_checkins',
                        'Date of Last ES Stay (Beta)':'date_of_last_stay',
                        'Date of First ES Stay (Beta)':'date_of_first_stay',
                        '4.2 Income Total at Entry':'income_at_entry',
                        '4.02 Total Income at Annual Update':'income_at_update',
                        '4.2 Income Total at Exit':'income_at_exit',
                        '4.3 Non-Cash Benefit Count':'non-cash_benefit_count',
                        '4.3 Non-Cash Benefit Count at Exit':'non-cash_count_at_exit',
                        'CurrentDate':'current_date',
                        'Barrier Count at Entry':'barrier_count_at_entry',
                        'Chronic Homeless Status':'chronic_homeless_status',
                        'ProgramType':'program_type',
                        'SOAR Eligibility Determination (Most Recent)':'soar_eligibility',
                        'SOAR Enrollment Determination (Most Recent)':'soar_enrollment',
                        'RRH | Most Recent Enrollment':'most_recent_rrh',
                        'Street Outreach | Most Recent Enrollment':'most_recent_street_outreach',
                        'Coordinated Entry | Most Recent Enrollment':'most_recent_CE',
                        'Emergency Shelter | Most Recent Enrollment':'most_recent_ES',
                        'Transitional Housing | Most Recent Enrollment':'most_recent_trans',
                        'PSH | Most Recent Enrollment':'most_recent_PSH',
                        'Prevention | Most Recent Enrollment':'most_recent_prevention',
                        'Under 25 Years Old':'under_25',
                        'ClientID':'client_id',
                        '4.10 Alcohol Abuse (Substance Abuse)':'alcohol_abuse',
                        '4.07 Chronic Health Condition':'chronic_health_condition',
                        '4.06 Developmental Disability':'developmental_disability',
                        '4.10 Drug Abuse (Substance Abuse)':'substance_abuse',
                        '4.08 HIV/AIDS':'HIV_AIDS',
                        '4.09 Mental Health Problem':'mental_health_problem',
                        '4.05 Physical Disability':'physical_disability',
                        'CaseChildren':'case_children','CaseAdults':'case_adults',
                        'Bed Nights During Report Period':'bednights_during_report_period',
                        'Count of Bed Nights - Entire Episode':'entire_episode_bednights',
                        'HEN-HP Referral Most Recent':'most_recent_HEN-HP',
                        'HEN-RRH Referral Most Recent':'most_recent_HEN-RRH',
                        'WorkSource Referral Most Recent':'most_recent_worksource',
                        'YAHP Referral Most Recent':'most_recent_YAHP',
                        '4.04.10 Other Public':'other_public',
                        '4.04.10 State Funded':'state_funded',
                        '4.04.11 Indian Health Services (IHS)':'indian_health_services',
                        '4.04.12 Other':'other',
                        '4.04.3 Combined Childrens HealthInsurance/Medicaid':'combined_childrens_health_insurance',
                        '4.04.3 Medicaid':'medicaid','4.04.4 Medicare':'medicare',
                        "4.04.5 State Children's health Insurance S-CHIP":'CHIP',
                        "4.04.6 Veteran's Administration Medical Services":'VAMS',
                        '4.04.8 Health Insurance obtained through COBRA':'COBRA',
                        '4.04.7 Private - Employer':'Private_employer',
                        '4.04.9 Private':'private','4.04.9 Private - Individual':'private_individual',
                        '4.2.3a Earned Income':'earned_income',
                        '4.2.4b Unemployment Insurance':'unemployement_income',
                        '4.2.5c Supplemental Security Income':'supplemental_security_income',
                        '4.2.6d Social Security Disability  Income':'social_security_disability_income',
                        '4.2.7e VA Disability Compensation':'VA_disability_compensation',
                        '4.2.8f VA Disability Pension':'VA_disability_pension',
                        '4.2.9g Private Disability Income':'private_disability_income',
                        '4.2.10h Workers Compensation':'workers_compensation',
                        '4.2.11i TANF':'TANF','4.2.12j General Assistance':'general_assistance',
                        '4.2.13k Retirement (Social Security)':'retirement_social_security',
                        '4.2.14l Pension from a Former Job':'pension_from_former_job',
                        '4.2.15m Child Support':'child_support',
                        '4.2.16n Alimony':'alimony',
                        '4.2.17o Other Income':'other_income',
                        'Chronic Homeless Status_vHMISDatAssessment':'chronic_homeless_status_assessment',
                        'Chronic Homeless Status_EvaluatevHMIS&HMISDA':'chronic_homeless_status_evaluation',
                        'Email':'email','HomePhone':'home_phone','WorkPhone':'work_phone'})

In [7]:
df.head()

,org_name,project_name,project_type,util_track_method,fed_grant_programs,enrollment_created_by,personal_id,case_id,relationship_to_HoH,client_location,...,pension_from_former_job,child_support,alimony,other_income,chronic_homeless_status_assessment,chronic_homeless_status_evaluation,email,home_phone,work_phone,perm_leaver
0,Family Promise of Spokane,FPS--ES--Bridges Family Shelter,Emergency shelter,Entry/Exit (e/e),NaN,Lynnetta Lewis,56550,92775,Son,NaN,...,0,0.0,0,0,0,0,NaN,NaN,NaN,False
1,Family Promise of Spokane,FPS--ES--Bridges Family Shelter,Emergency shelter,Entry/Exit (e/e),NaN,Lynnetta Lewis,36138,92775,Self,Spokane City & County CoC,...,0,0.0,0,0,0,0,NaN,509-847-3635,NaN,False
2,Family Promise of Spokane,FPS--ES--Bridges Family Shelter,Emergency shelter,Entry/Exit (e/e),NaN,Lynnetta Lewis,36139,92775,Daughter,NaN,...,0,0.0,0,0,0,0,NaN,NaN,NaN,False
3,Family Promise of Spokane,FPS--ES--Bridges Family Shelter,Emergency shelter,Entry/Exit (e/e),NaN,Lynnetta Lewis,36140,92775,Son,NaN,...,0,0.0,0,0,0,0,NaN,NaN,NaN,False
4,Family Promise of Spokane,FPS--ES--Bridges Family Shelter,Emergency shelter,Entry/Exit (e/e),NaN,Lynnetta Lewis,36141,92775,Significant Other (Non-Married),NaN,...,0,0.0,0,0,1,1,NaN,964-8982,NaN,False


In [8]:
train = df

# Split train into train & val
train, test = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['perm_leaver'], random_state=42)

In [9]:
# Hand pick features only known at entry to avoid data leakage
features = ['case_members','ssn_quality','dob_quality',
            'age_at_enrollment','race','ethnicity',
            'gender','vet_status','disabling_cond_at_entry',
            'zip','covered_by_health_insurance','domestic_violence',
            'currently_fleeing','in_school','sexual_orientation',
            'last_grade_completed','employed_status','looking_for_work',
            'general_health_status','dental_health_status','mental_health_status',
            'pregnancy_status','income_at_entry','barrier_count_at_entry',
            'chronic_homeless_status','soar_eligibility','soar_enrollment',
            'under_25','alcohol_abuse','chronic_health_condition',
            'developmental_disability','substance_abuse','mental_health_problem',
            'physical_disability','case_children','case_adults','indian_health_services'
             ]

In [10]:
target = 'perm_leaver'
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [11]:
y_train

586     False
54       True
1080    False
276     False
1305    False
        ...  
122      True
385     False
1244     True
1085     True
27       True
Name: perm_leaver, Length: 1524, dtype: bool

In [12]:
# try my decision tree

import category_encoders as ce
from sklearn.impute import SimpleImputer

from sklearn.pipeline import make_pipeline


# Make pipeline!
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='most_frequent'), 
    good_tree.DecisionTreeClassifier(max_depth=1) 
                       )


# Fit on train, score on val
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print('Validation Accuracy', accuracy_score(y_test, y_pred))

Validation Accuracy 0.6902887139107612


In [13]:
# test other decision tree

import category_encoders as ce
from sklearn.impute import SimpleImputer

from sklearn.pipeline import make_pipeline


# Make pipeline!
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='most_frequent'), 
    RandomForestClassifier() 
                       )


# Fit on train, score on val
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print('Validation Accuracy', accuracy_score(y_test, y_pred))

Validation Accuracy 0.7191601049868767


In [14]:
"""RandomForest was approximately 3 percent more accurate than my decision tree

Random Forest can be tuned in many more ways, clearly making it better than my model,
but my basic implementation with a max_depth arg was able to do fairly well.

"""

'RandomForest was approximately 3 percent more accurate than my decision tree\n\nRandom Forest can be tuned in many more ways, clearly making it better than my model,\nbut my basic implementation with a max_depth arg was able to do fairly well.\n\n'